### Get 1h temporary access_token

In [1]:
import requests, base64, json, re
client_id = "897e6482239c46b89683f64d01a80570"
client_secret = input()

c:\Users\Tim\AppData\Local\Programs\Python\Python38\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


In [2]:
res = requests.post(url="https://accounts.spotify.com/api/token",
                    headers={
                        'Authorization': 'Basic ' + base64.b64encode(str.encode(client_id + ":" + client_secret)).decode(),
                        "Content-Type": "application/x-www-form-urlencoded"
                    },
                    data={
                        "grant_type": "client_credentials"
                    })
res.text
access_token = eval(res.text)["access_token"]

## download list of Playlists to fetch content from

In [3]:
# search for playlists matching "Top Hits of ****"
from urllib import parse
playlists = []
for year in range(1970, 2023, 1):
  name = "Top Hits of " + str(year)
  name_2 = "Top Tracks of " + str(year)
  q = requests.get(url="https://api.spotify.com/v1/search",
                headers = {"Authorization": "Bearer " + access_token},
                params={
                  "q": "name:" + name,
                  "type": "playlist",
                  "market": "US",
                  "limit": 10,
                  "offset": 0
                })

  j = json.loads(q.text)
  for found_playlist in j["playlists"]["items"]:
      if found_playlist["name"] == name or found_playlist["name"] == name_2:
          if found_playlist["owner"]["id"] == "spotify":
            id = re.search("playlists\/(\w{22})", found_playlist["href"]).group(1)
            title = found_playlist["name"]
            playlists.append((id, title))

In [4]:
len(playlists)

51

In [6]:
playlists[-4:] # nothing after 2020 found

[('37i9dQZF1DWTE7dVUebpUW', 'Top Hits of 2017'),
 ('37i9dQZF1DXe2bobNYDtW8', 'Top Hits of 2018'),
 ('37i9dQZF1DWVRSukIED0e9', 'Top Hits of 2019'),
 ('37i9dQZF1DX7Jl5KP2eZaS', 'Top Tracks of 2020')]

# Get Songs and Features from Playlists

In [7]:
track_ids = []
for playlist in playlists:
  q = requests.get(url="https://api.spotify.com/v1/playlists/" + playlist[0] + "/tracks",
                #headers = h,
                headers = {"Authorization": "Bearer " + access_token},
                params={
                  "fields": "items(track.href)",
                  #"market": "US",
                  #"limit": 100, # check if any playlist has more than 100! increase!
                  "offset": 0
                })
  j = json.loads(q.text)
  for track in j["items"]:
      id = re.search("tracks\/(\w{22})", track["track"]["href"]).group(1)
      track_ids.append(id)

In [8]:
len(track_ids)

5050

In [9]:
tracks = []
for id in track_ids:
    # Get basic data
    q_main = requests.get(url="https://api.spotify.com/v1/tracks/" + id,
        #headers = h,
        headers = {"Authorization": "Bearer " + access_token},
        # params={
        #   "market": "US",
        # }
        )
    j_main = json.loads(q_main.text)
    # with open("data/" + str(id) + '.json', 'w') as f:
    #     json.dump(j, f)
    # with open("data/" + str(id) + '.json', 'w', encoding='utf-8') as f:
    #     json.dump(j_main, f, ensure_ascii=False, indent=4)

    # get track audio features
    q_features = requests.get(url="https://api.spotify.com/v1/audio-features/" + id,
        #headers = h,
        headers = {"Authorization": "Bearer " + access_token},
        # params={
        #   "market": "US",
        # }
        )
    j_features = json.loads(q_features.text)
    # with open("data/" + str(id) + '.json', 'w') as f:
    #     json.dump(j, f)
    data_combined = {**j_main,**j_features}
    with open("data/" + str(id) + '.json', 'w', encoding='utf-8') as f:
        json.dump(data_combined, f, ensure_ascii=False, indent=4)